**Tabla de contenido**

- [Introducción](#Introducción)
- [Lib](#Lib)
- [Lectura de datos](#Lectura-de-datos)
- [Feature engineering](#Feature-engineering)
- [Exportar CSV](#Exportar-CSV)

# Introducción

En este cuaderno, construiremos un dataset que nos permitirá realizar predicciones dentro de un rango de fechas determinado.

# Lib

Para garantizar el correcto funcionamiento de este cuaderno, se requieren las siguientes librerías:

- **Python:** Versión 3.12.3
- **Pandas:** Versión 2.2.3
- **NumPy:** Versión 1.26.0

In [1]:
import os
import pandas as pd
import numpy as np

# Lectura de datos

Los datos fueron leídos desde el archivo **WalmartSalesClean.csv**, que fue exportado desde el cuaderno LDV-APV y utiliza el carácter **","** como delimitador.

**Para leer correctamente este archivo, es necesario proporcionar la ruta donde se encuentra almacenado.**

In [2]:
os.chdir('D:\\6. NEXER\\master\\Solucion TFM\\2. Datos en CSV')
WalmartSales = pd.read_csv('WalmartSalesClean.csv', delimiter=',')
WalmartSales.head(2)

,Date,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Holiday_Events
0,2010-02-12,1,164195744,1,38.51,2.548,211.242176,8.106,1
1,2010-02-19,1,161196816,0,39.93,2.514,211.289139,8.106,0


# Feature engineering

Vamos a seleccionar las variables necesarias.

In [3]:
WalmartSales['Date'] = pd.to_datetime(WalmartSales['Date'],format = '%Y-%m-%d')
WalmartSales['week'] = WalmartSales['Date'].dt.isocalendar().week
WalmartSales = WalmartSales[['Store','CPI','Unemployment','Holiday_Events','week','Date']]
WalmartSales = WalmartSales.set_index('Date',drop=True)
WalmartSales.head()

,Store,CPI,Unemployment,Holiday_Events,week
Date,,,,,
2010-02-12,1,211.242176,8.106,1,6
2010-02-19,1,211.289139,8.106,0,7
2010-02-26,1,211.319642,8.106,0,8
2010-03-05,1,211.350144,8.106,0,9
2010-03-12,1,211.380646,8.106,0,10


Para cada tienda, consultemos el CPI y Unemployment  mínimo y máximo.

In [4]:
df = WalmartSales.copy()
dicc = {
        'Store':[],
        'CPI_min':[],
        'CPI_max':[],
        'Unemployment_min':[],
        'Unemployment_max':[]
        }
for store in df['Store'].unique():
    dicc['Store'].append(store)
    dicc['CPI_min'].append(df[df['Store'] == store]['CPI'].min())
    dicc['CPI_max'].append(df[df['Store'] == store]['CPI'].max())
    dicc['Unemployment_min'].append(df[df['Store'] == store]['Unemployment'].min())
    dicc['Unemployment_max'].append(df[df['Store'] == store]['Unemployment'].max())


df_dicc = pd.DataFrame(dicc)
df_dicc.head(45)

,Store,CPI_min,CPI_max,Unemployment_min,Unemployment_max
0,1,210.339968,223.444250,6.573,8.106
1,2,209.998464,223.078349,6.170,8.324
2,3,213.619610,226.968832,6.034,7.574
3,4,126.064000,131.193101,3.879,8.623
4,5,210.887283,224.037811,5.422,6.768
5,6,211.844275,225.068621,5.329,7.259
6,7,189.381696,199.219533,7.557,9.137
7,8,213.665536,227.036928,5.124,6.433
8,9,213.848480,227.232819,4.954,6.560
9,10,126.076646,131.193101,6.943,9.765


Generaremos las variables con valores aleatorios dentro del rango definido por los valores mínimos y máximos.

El rango de fechas abarca desde el 26 de octubre de 2012 hasta el 28 de diciembre de 2012, con una frecuencia semanal cada viernes.

In [5]:
# generación de las variables
variables ={
    'Date':[],
    'Store':[],
    'CPI':[],
    'Unemployment':[],
    'Holiday_Events':[]
    }
for store in df_dicc['Store']:
    variables['Store'].extend([store]*10) # repite la tienda 10 veces
    variables['CPI'].extend(np.random.uniform(df_dicc[df_dicc['Store']==store]['CPI_min'],
                                              df_dicc[df_dicc['Store']==store]['CPI_max'],10))
    variables['Unemployment'].extend(np.random.uniform(df_dicc[df_dicc['Store']==store]['Unemployment_min'],
                                                       df_dicc[df_dicc['Store']==store]['Unemployment_max'],10))
    variables['Holiday_Events'].extend(np.random.randint(0,1,10))
    variables['Date'].extend(pd.date_range(start='2012-10-26',end='2012-12-28',freq='W-FRI'))
variables_df = pd.DataFrame(variables)
variables_df['week'] = variables_df['Date'].dt.isocalendar().week
variables_df = variables_df.set_index('Date',drop=True)
variables_df.head(20)

,Store,CPI,Unemployment,Holiday_Events,week
Date,,,,,
2012-10-26,1,220.060338,6.751995,0,43
2012-11-02,1,214.707935,7.194075,0,44
2012-11-09,1,214.420035,7.883895,0,45
2012-11-16,1,221.387569,7.772605,0,46
2012-11-23,1,216.973749,7.035673,0,47
2012-11-30,1,213.863766,7.182224,0,48
2012-12-07,1,219.036550,7.310641,0,49
2012-12-14,1,217.779235,6.832031,0,50
2012-12-21,1,220.439491,6.996928,0,51


# Exportar CSV

Guardemos nuestro dataset en un archivo **.csv**.

In [6]:
variables_df.to_csv('Dataset_nuevas_observaciones.csv')